In [1]:
import tensorflow as tf
import os
import numpy as np
import time
import matplotlib.pyplot as plt 
from cnn_model import model

In [2]:
from prep import mnist_funct,fashion_funct
mnist = mnist_funct()
fashion = fashion_funct()

19 10
19 10


In [3]:
mnist.keys(),fashion.keys()

(dict_keys(['train_x', 'train_y', 'test_x', 'test_y']),
 dict_keys(['train_x', 'train_y', 'test_x', 'test_y']))

In [4]:
mnist['train_x'].shape

(60000, 28, 28, 1)

### fashion 



In [5]:
labels = {10:'T-shirt/top',11:'Trouser',12:'Pullover',13:'Dress',14:'Coat',15:'Sandal',16:'Shirt',17:'Sneaker',18:'Bag',19:'Ankle boot'}

In [6]:
for i in range(10):
    labels[i] = str(i)

In [7]:
labels

{10: 'T-shirt/top',
 11: 'Trouser',
 12: 'Pullover',
 13: 'Dress',
 14: 'Coat',
 15: 'Sandal',
 16: 'Shirt',
 17: 'Sneaker',
 18: 'Bag',
 19: 'Ankle boot',
 0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9'}

### set batch_size

In [8]:
batch_size = 100

### train and test for cummulative dataset (shuffled) 

In [9]:
#train
dtrain_x = tf.data.Dataset.from_tensor_slices(np.vstack((mnist['train_x'],fashion['train_x'])))
dtrain_y = tf.data.Dataset.from_tensor_slices(np.hstack((mnist['train_y'],fashion['train_y']))).map(lambda x: tf.one_hot(x,20),num_parallel_calls=4)
train_data = tf.data.Dataset.zip((dtrain_x,dtrain_y)).shuffle(1000).repeat().batch(batch_size)
#one shot train
train_one_shot = tf.data.Dataset.zip((dtrain_x,dtrain_y)).batch(batch_size)
#test
dtest_x = tf.data.Dataset.from_tensor_slices(np.vstack((mnist['test_x'],fashion['test_x'])))
dtest_y = tf.data.Dataset.from_tensor_slices(np.hstack((mnist['test_y'],fashion['test_y']))).map(lambda x: tf.one_hot(x,20),num_parallel_calls=4)
test_data = tf.data.Dataset.zip((dtest_x,dtest_y)).shuffle(1000).batch(batch_size)
#valid mnist
dvalid_mnist_x = tf.data.Dataset.from_tensor_slices(mnist['test_x'][:1001])
dvalid_mnist_y = tf.data.Dataset.from_tensor_slices(mnist['test_y'][:1001]).map(lambda x: tf.one_hot(x,20),num_parallel_calls=4)
valid_mnist_data =  tf.data.Dataset.zip((dvalid_mnist_x,dvalid_mnist_y)).repeat().batch(batch_size)
#valid fashion
dvalid_fashion_x = tf.data.Dataset.from_tensor_slices(fashion['test_x'][:1001])
dvalid_fashion_y = tf.data.Dataset.from_tensor_slices(fashion['test_y'][:1001]).map(lambda x: tf.one_hot(x,20),num_parallel_calls=4)
valid_fashion_data =  tf.data.Dataset.zip((dvalid_fashion_x,dvalid_fashion_y)).repeat().batch(batch_size)

(train_data,test_data,valid_mnist_data,valid_fashion_data)

(<BatchDataset shapes: ((?, 28, 28, 1), (?, 20)), types: (tf.float32, tf.float32)>,
 <BatchDataset shapes: ((?, 28, 28, 1), (?, 20)), types: (tf.float32, tf.float32)>,
 <BatchDataset shapes: ((?, 28, 28, 1), (?, 20)), types: (tf.float32, tf.float32)>,
 <BatchDataset shapes: ((?, 28, 28, 1), (?, 20)), types: (tf.float32, tf.float32)>)


### individual test datasets

In [10]:
#mnist train
mnist_train_x = tf.data.Dataset.from_tensor_slices(mnist['train_x'])
mnist_train_y = tf.data.Dataset.from_tensor_slices(mnist['train_y']).map(lambda x: tf.one_hot(x,20),num_parallel_calls=4)
mnist_train = tf.data.Dataset.zip((mnist_train_x,mnist_train_y)).batch(batch_size)
#mnist test
mnist_test_x = tf.data.Dataset.from_tensor_slices(mnist['test_x'])
mnist_test_y = tf.data.Dataset.from_tensor_slices(mnist['test_y']).map(lambda x: tf.one_hot(x,20),num_parallel_calls=4)
mnist_test = tf.data.Dataset.zip((mnist_test_x,mnist_test_y)).batch(batch_size)
#fashion train
fashion_train_x = tf.data.Dataset.from_tensor_slices(fashion['train_x'])
fashion_train_y = tf.data.Dataset.from_tensor_slices(fashion['train_y']).map(lambda x: tf.one_hot(x,20),num_parallel_calls=4)
fashion_train = tf.data.Dataset.zip((fashion_train_x,fashion_train_y)).batch(batch_size)
#fashion test
fashion_test_x = tf.data.Dataset.from_tensor_slices(fashion['test_x'])
fashion_test_y = tf.data.Dataset.from_tensor_slices(fashion['test_y']).map(lambda x: tf.one_hot(x,20),num_parallel_calls=4)
fashion_test =tf.data.Dataset.zip((fashion_test_x,fashion_test_y)).batch(batch_size)

In [11]:
# data pipline iterator initializer
iterator = tf.data.Iterator.from_structure(train_data.output_types,train_data.output_shapes)
get_batch = iterator.get_next()

#train data initiallizer shuffled
train_init = iterator.make_initializer(train_data)
#test data initializer shuffled
test_init = iterator.make_initializer(test_data)
#train one shot
train_1s_init = iterator.make_initializer(train_one_shot)
# valid data mnist
valid_mnist_init = iterator.make_initializer(valid_mnist_data)
# valid data fashion
valid_fashion_init = iterator.make_initializer(valid_fashion_data)
#fashoin train data initializer
fashion_train_init = iterator.make_initializer(fashion_train)
# fashion test data initializer 
fashion_test_init = iterator.make_initializer(fashion_test)
#mnist train data initializer
mnist_train_init = iterator.make_initializer(mnist_train)
#mnist test data initializer
mnist_test_init = iterator.make_initializer(mnist_test)

In [12]:
param=[]
param.append(1)
param.append([5,64])
param.append([5,64])
param.append(1000)
param.append(20)

In [13]:
is_train = tf.placeholder(shape=(),dtype=tf.bool)
prob_keep = tf.placeholder(shape=(),dtype=tf.float32)

In [14]:
cnn = model(get_batch[0],is_train,prob_keep,param)

In [15]:
logits = cnn.logits
info = cnn.total_params()

varname : conv_1/weight:0 [Dimension(5), Dimension(5), Dimension(1), Dimension(64)]
varname : conv_1/bias:0 [Dimension(64)]
varname : conv_2/weight:0 [Dimension(5), Dimension(5), Dimension(64), Dimension(64)]
varname : conv_2/bias:0 [Dimension(64)]
varname : dense/kernel:0 [Dimension(3136), Dimension(1000)]
varname : dense/bias:0 [Dimension(1000)]
varname : dense_1/kernel:0 [Dimension(1000), Dimension(20)]
varname : dense_1/bias:0 [Dimension(20)]
total number of trainable parameter : 3261148


In [16]:
logits

<tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 20) dtype=float32>

In [17]:

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=get_batch[1],logits=logits))

tf.summary.scalar("losses",loss)
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

#get accuracy
predictions = tf.argmax(logits,axis=1)
equality = tf.equal(predictions,tf.argmax(get_batch[1],axis=1))
accuracy = tf.reduce_mean(tf.cast(equality,tf.float32))
tf.summary.scalar("accuracy",accuracy)
init_op = tf.global_variables_initializer()

In [18]:
log_dir = '/output/logs'
print(log_dir)

/output/logs


In [19]:
t_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(log_dir)
saver = tf.train.Saver()

In [20]:
epochs = 1
epochs = int(epochs*120000/batch_size)
probability_keep = 0.5

In [ ]:
duration = 0
log_mnist_test = 0
log_fashion_test = 0
with tf.Session() as sess:
    #saver.restore(sess,log_dir+'/curr_model.ckpt')
    writer.add_graph(sess.graph)
    sess.run(init_op)
    train_log_loss = []
    train_log_mnist_acc = []
    train_log_fashion_acc = []
    
    for i in range(1,epochs+1):
        sess.run(train_init)
        start_time = time.time()
        l,_,acc,s = sess.run([loss,optimizer,accuracy,t_summary],feed_dict={is_train:True,prob_keep:probability_keep})
        duration = duration + time.time() - start_time
        writer.add_summary(s,i)
        # history of loss and accuracy while training
        train_log_loss.append(l)
        print("running epoch  : {}".format(i))
        if i%100==0:
            print("Epoch :{}, loss :{:.3f}, accuracy :{:.3f}".format(i,l,acc))
            save_path = saver.save(sess,log_dir+'/curr_model.ckpt')
            
            test_iter = 10
            mnist_avg_acc = 0
            sess.run(valid_mnist_init)
            for _ in range(test_iter):
                acc = sess.run([accuracy],feed_dict = {is_train:False,prob_keep:1.0})
                mnist_avg_acc+=acc[0]
            mnist_avg_acc=mnist_avg_acc*100.0/test_iter
            print("validation mnist accuracy :{:.3f}".format(mnist_avg_acc))
            train_log_mnist_acc.append(mnist_avg_acc)
        
            fashion_avg_acc = 0
            sess.run(valid_fashion_init)
            for _ in range(test_iter):
                acc = sess.run([accuracy],feed_dict = {is_train:False,prob_keep:1.0})
                fashion_avg_acc+=acc[0]
            fashion_avg_acc=fashion_avg_acc*100.0/test_iter
            print("validation fashion accuracy :{:.3f}".format(fashion_avg_acc))
            train_log_fashion_acc.append(fashion_avg_acc)
    
    #final accuracy on test data
    i=0
    t_acc = 0
    sess.run(test_init)
    while True:
        try:
            i+=1
            #100 --> batch size for one_shot_iter is 100
            acc = sess.run([accuracy],feed_dict = {is_train:False,prob_keep:1.0})
            t_acc+=acc[0]
        except tf.errors.OutOfRangeError:
            i = i-1
            print("Test Accuracy: {}".format(t_acc*100.0/(i)))
            total_test_acc = t_acc*100.0/i
            plt.plot(train_log_loss,label='loss_train',color='r')
            plt.plot(train_log_mnist_acc,label='acc_mnist',color='b')
            plt.plot(train_log_fashion_acc,label='acc_fashion',color='g')
            plt.xlabel("epochs")
            plt.legend()
            plt.show()
            break
    
    #total train accuracy
    i=0
    t_acc = 0
    sess.run(train_1s_init)
    while True:
        try:
            i+=1
            #100 --> batch size for one_shot_iter is 100
            acc = sess.run([accuracy],feed_dict = {is_train:False,prob_keep:1.0})
            t_acc+=acc[0]
        except tf.errors.OutOfRangeError:
            i = i-1
            print("total train Accuracy: {}".format(t_acc*100.0/(i)))
            total_train_acc = t_acc*100.0/i
            break
    
    
    
    #accuracy for mnist data
    i=0
    t_acc = 0
    sess.run(mnist_test_init)
    while True:
        try:
            i+=1
            #100 --> batch size for one_shot_iter is 100
            acc = sess.run([accuracy],feed_dict = {is_train:False,prob_keep:1.0})
            t_acc+=acc[0]
        except tf.errors.OutOfRangeError:
            i = i-1
            log_mnist_test = t_acc*100/i
            print("MNIST Test Accuracy: {}".format(t_acc*100/(i)))
            break
    #train mnist
    i=0
    t_acc = 0
    sess.run(mnist_train_init)
    while True:
        try:
            i+=1
            #100 --> batch size for one_shot_iter is 100
            acc = sess.run([accuracy],feed_dict = {is_train:False,prob_keep:1.0})
            t_acc+=acc[0]
        except tf.errors.OutOfRangeError:
            i = i-1
            log_mnist_train = t_acc*100/i
            print("MNIST train Accuracy: {}".format(t_acc*100/(i)))
            break
    
    #accuracy for fashion data
    i=0
    t_acc = 0
    sess.run(fashion_test_init)
    while True:
        try:
            i+=1
            #100 --> batch size for one_shot_iter is 100
            acc = sess.run([accuracy],feed_dict = {is_train:False,prob_keep:1.0})
            t_acc+=acc[0]
        except tf.errors.OutOfRangeError:
            i = i-1
            log_fashion_test = t_acc*100/i
            print("fashion test Accuracy: {}".format(t_acc*100.0/(i)))
            break
    # train
    i=0
    t_acc = 0
    sess.run(fashion_train_init)
    while True:
        try:
            i+=1
            #100 --> batch size for one_shot_iter is 100
            acc = sess.run([accuracy],feed_dict = {is_train:False,prob_keep:1.0})
            t_acc+=acc[0]
        except tf.errors.OutOfRangeError:
            print("i : "+str(i))
            i = i-1
            log_fashion_train = t_acc*100/i
            print("fashion Train Accuracy: {}".format(t_acc*100.0/(i)))
            break


running epoch  : 1
running epoch  : 2
running epoch  : 3
running epoch  : 4
running epoch  : 5
running epoch  : 6
running epoch  : 7
running epoch  : 8
running epoch  : 9
running epoch  : 10
running epoch  : 11
running epoch  : 12
running epoch  : 13
running epoch  : 14
running epoch  : 15
running epoch  : 16
running epoch  : 17
running epoch  : 18
running epoch  : 19
running epoch  : 20
running epoch  : 21
running epoch  : 22
running epoch  : 23
running epoch  : 24
running epoch  : 25
running epoch  : 26
running epoch  : 27
running epoch  : 28
running epoch  : 29
running epoch  : 30
running epoch  : 31
running epoch  : 32
running epoch  : 33
running epoch  : 34
running epoch  : 35
running epoch  : 36
running epoch  : 37
running epoch  : 38
running epoch  : 39
running epoch  : 40
running epoch  : 41
running epoch  : 42
running epoch  : 43
running epoch  : 44
running epoch  : 45
running epoch  : 46
running epoch  : 47
running epoch  : 48
running epoch  : 49
running epoch  : 50
running e

In [ ]:
# save log files
log=dict()
log['batch_size'] = batch_size
log['epochs'] = epochs
log['param_info'] = info
log['duration'] = np.array(duration)
log['loss'] = np.array(train_log_loss)
log['total_train_acc'] = total_train_acc
log['total_test_acc'] = total_test_acc
log['mnist_valid'] = np.array(train_log_mnist_acc)
log['fashion_valid'] = np.array(train_log_fashion_acc)
log['mnist_train_test'] = np.array([log_mnist_train,log_mnist_test])
log['fashion_train_test'] = np.array([log_fashion_train,log_fashion_test])
np.save('/output/proj_shuffle_1e.npy',log)